In [1]:

%load_ext autoreload
%autoreload 2

In [5]:
import sys
from pathlib import Path

project_root = Path().resolve().parents[1]


In [ ]:
import os
import re

import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from io import StringIO
import pandas as pd
import pickle
import igraph as ig

from dataprep.util import create_pattern_dict, filter_dict
from dataprep.util import *
from dataprep.plotting_helpers import *
from dataprep.generation import *
from util import *

# === CONFIG ===
current_dir = os.path.dirname(os.getcwd())
FILE_PATH = os.path.join(current_dir, "data", "IBM", "HI-SmallTransactions.txt")

current_dir = os.path.dirname(os.getcwd())
CSV_PATH = os.path.join(current_dir, "data", "IBM", "Hi-Small_Trans.csv")

pattern_dict = create_pattern_dict(FILE_PATH)

bipartite_data = filter_dict(pattern_dict, "BIPARTITE" )  # Example usage function

df_data = pd.DataFrame(bipartite_data, columns=columns)
df_data['src'] = df_data['From Bank'] + "_" + df_data['From Account']
df_data['dst'] = df_data['To Bank'] + "_" + df_data['To Account']

df_full = pd.read_csv(CSV_PATH, dtype=str)
df_full["Is Laundering"] = pd.to_numeric(df_full["Is Laundering"], errors="coerce").fillna(2).astype(int)
df = df_full[df_full['Is Laundering'] ==1]

df_full["From_Node"] = df_full["From Bank"].astype(str) + "_" + df_full["Account"].astype(str)
df_full["To_Node"] = df_full["To Bank"].astype(str) + "_" + df_full["Account.1"].astype(str)

# full_data["Amount Paid"] = full_data["Amount Paid"].astype(float)
# full_data["Amount Received"] = full_data["Amount Received"].astype(float)

df["From_Node"] = df["From Bank"].astype(str) + "_" + df["Account"].astype(str)
df["To_Node"] = df["To Bank"].astype(str) + "_" + df["Account.1"].astype(str)

START_ACCOUNT = "0222_812D127D0"


ImportError: attempted relative import with no known parent package

In [ ]:
def process_all_suspicious_networks_igraph(df):
    """
    Iteratively discovers suspicious networks without reusing nodes
    using igraph for fast traversal.
    """

    # Build graph
    g = ig.Graph.DataFrame(df[["From_Node", "To_Node"]], directed=True)

    # Map node name → vertex index
    name_to_vid = {name: vid for vid, name in enumerate(g.vs["name"])}

    unvisited_vids = set(range(g.vcount()))
    global_seen_vids = set()

    networks = []

    while unvisited_vids:
        start_vid = unvisited_vids.pop()

        if start_vid in global_seen_vids:
            continue

        # Fast BFS / component expansion
        reachable_vids = set(g.bfs(start_vid)[0])

        if not reachable_vids:
            global_seen_vids.add(start_vid)
            continue

        # Convert vids → node names
        connected_nodes = set(g.vs[list(reachable_vids)]["name"])

        # Filter df to only edges fully inside this subgraph
        connected_df = df[
            df["From_Node"].isin(connected_nodes)
            & df["To_Node"].isin(connected_nodes)
        ]

        # Store result
        networks.append({
            "start_node": g.vs[start_vid]["name"],
            "nodes": connected_nodes,
            "transactions": connected_df
        })

        # Mark globally seen
        global_seen_vids.update(reachable_vids)
        unvisited_vids.difference_update(reachable_vids)

        print(
            f"Network from {g.vs[start_vid]['name']}: "
            f"{len(connected_nodes)} nodes, "
            f"{len(connected_df)} transactions"
        )

    return networks


In [ ]:
df = df[df['Is Laundering'] ==1]

networks = process_all_suspicious_networks_igraph(df)

NameError: name 'ig' is not defined